## Comprobación del funcionamiento en el dataset completo:

He localizado el dataset completo en Kaggle, y he comprobado mi regla y sistema, los resultados son muy buenos.

Sí nosotros teniamos 4 días, el dataset completo es un mes.

Entreno un randomforrest con las 96 primeros steps, respentando el 20 % para test. Y despúes predigo con ese modelos el resto del dataset, del step 97 al 743.
Suponen 5.303.817 casos, donde existen 7.061 casos de fraude.

Los resultados son espectaculares, detecta 7.010 casos de 7.061 casos de fraude que existen en esos steps que no participaron en el entrenamiento.

La regla ( t_c) le aporta 6916 acierto, y el  randomforest mejora el aciero hasta los 7.010 aciertos. No se genera ningun falso negativo.

El autor de paysim tiene que mejorar como se genera esta base de datos sintetica o simplemente con la regla se consiguen muy buenso resultados. 

Este cuaderno se tiene que ejecutar en Google Colab, ya que, al menos mi ordenador personal, no soporta el tamaño del dataset.

Acceso colab: https://colab.research.google.com/drive/1DWe1MaSk3dBXWjkDUnQGv894fqQTsnyH?usp=sharing 








In [ ]:
from google.colab import files
import os

In [ ]:
# Montamos nuestro drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install category_encoders


In [ ]:
import numpy as np
# primero todas las librerías por bloques

import pandas as pd

warnings.filterwarnings("ignore")

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split


import category_encoders as ce

from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, \
                            recall_score, precision_score, make_scorer, \
                            roc_auc_score, f1_score, precision_recall_curve, \
                            accuracy_score,plot_confusion_matrix,average_precision_score


from sklearn import metrics










In [ ]:
os.chdir('/content/drive/MyDrive/amex_k')

In [ ]:

data=pd.read_csv('/content/drive/MyDrive/amex_k/PS_20174392719_1491204439457_log.csv',delimiter=',')

In [ ]:
data.shape

(6362620, 11)

In [ ]:
data.columns

Index(['step', 'type', 'amount', 'nameOrig', 'oldbalanceOrg', 'newbalanceOrig',
       'nameDest', 'oldbalanceDest', 'newbalanceDest', 'isFraud',
       'isFlaggedFraud'],
      dtype='object')

In [ ]:
dat=data[['step','type', 'amount', 'nameOrig', 'oldbalanceOrg', 'newbalanceOrig',
       'nameDest', 'oldbalanceDest', 'newbalanceDest', 'isFraud']]

In [ ]:
dat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 10 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   type            object 
 2   amount          float64
 3   nameOrig        object 
 4   oldbalanceOrg   float64
 5   newbalanceOrig  float64
 6   nameDest        object 
 7   oldbalanceDest  float64
 8   newbalanceDest  float64
 9   isFraud         int64  
dtypes: float64(5), int64(2), object(3)
memory usage: 485.4+ MB


In [ ]:
tipo_origen=dat.nameOrig.apply(lambda x: "".join([y[0] for y in (x.split())]))
tipo_destino=dat.nameDest.apply(lambda x: "".join([y[0] for y in (x.split())]))
dat['segmento']=tipo_origen+tipo_destino

In [ ]:
dat=dat.drop(['nameOrig','nameDest'],axis=1)

In [ ]:
def check_t_c(df):
  '''
    Controla si las operacioens son Transfer o Cash_out y si el importe 
    es igual a todo el oldbalanceorig de la cuenta origen. Vaciado de cuentas.

    Args:
    param1: Pandas Df de datos.
        
    Returns:
    Int 0,1 reperesentando valores booleanos.
   '''


  cond_1= (df['type'] == 'CASH_OUT')
  cond_2= (df['type'] == 'TRANSFER')
  cond_3= (df['amount'] == df['oldbalanceOrg'])
  cond_final=((cond_1 | cond_2) & cond_3)
  return np.multiply(cond_final,1)

In [ ]:
dat['t_c']=dat.apply(check_t_c,axis=1)

In [ ]:
dat['step_order']=dat.groupby('step').cumcount()+1

In [ ]:
dat['step_importance']=np.power(dat['step_order'].astype(float),-1)

In [ ]:
dat.columns

Index(['step', 'type', 'amount', 'oldbalanceOrg', 'newbalanceOrig',
       'oldbalanceDest', 'newbalanceDest', 'isFraud', 'segmento', 't_c',
       'step_order', 'step_importance'],
      dtype='object')

In [ ]:
map_sig_orig={'CASH_OUT':-1,
         'TRANSFER':-1,
         'PAYMENT': -1,
         'CASH_IN': 1,
         'DEBIT': -1
         }
map_sig_dest={'CASH_OUT':-1,
         'TRANSFER': +1,
         'PAYMENT': 1,
         'CASH_IN': -1,
         'DEBIT': 1
         }

In [ ]:
dat['errorbalanceorig'] = dat['type'].map(map_sig_orig)*(dat.newbalanceOrig - dat.oldbalanceOrg)  - dat.amount
dat['errorbalancedest'] = dat['type'].map(map_sig_dest)*(dat.oldbalanceDest - dat.newbalanceDest)  + dat.amount

In [ ]:
selected_feaures_target=['step','oldbalanceOrg'  ,'newbalanceDest' , 'step_importance', 't_c', 'errorbalanceorig', 'errorbalancedest','isFraud']

In [ ]:
dat=dat[selected_feaures_target]
dat.columns= dat.columns.str.lower()

In [ ]:
dat.columns

Index(['step', 'oldbalanceorg', 'newbalancedest', 'step_importance', 't_c',
       'errorbalanceorig', 'errorbalancedest', 'isfraud'],
      dtype='object')

In [ ]:
entrenamiento=dat[dat.step<97]

In [ ]:
X=entrenamiento.drop(['isfraud'],axis=1)
y=entrenamiento['isfraud']

In [ ]:
# Para randon forrest no hace falta ni escalar, ni hacer Onehotencoder, los arboles funcionan igual.

X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.20,random_state= 42,stratify=y)

In [ ]:
model_rfc=RandomForestClassifier(n_estimators= 100, max_features='sqrt', max_depth=8, criterion='gini',class_weight='balanced')

model_rfc.fit(X_train,y_train)

RandomForestClassifier(class_weight='balanced', max_depth=8,
                       max_features='sqrt')

In [ ]:
X_out_sample=dat[dat.step>=97].drop(['isfraud'],axis=1)
y_out_sampe= dat[dat.step>=97]['isfraud']

In [ ]:
y_pred=model_rfc.predict(X_out_sample)

In [ ]:
def evaluate_model(name,y_test, y_pred, ypred_proba = None):
  if ypred_proba is not None:
      print('ROC-AUC score of the model: {}'.format(roc_auc_score(y_test, ypred_proba[:, 1])))
  print('Accuracy of the model: {}\n'.format(accuracy_score(y_test, y_pred)))
  print('Classification report: \n{}\n'.format(classification_report(y_test, y_pred)))
  print('Confusion matrix: \n{}\n'.format(confusion_matrix(y_test, y_pred)))
  print('Normalized confusion matrix: \n{}\n'.format(confusion_matrix(y_test, y_pred,normalize='all')))

  print("Average Precision-Recall score:\t", average_precision_score(y_test, y_pred,average='macro'))

  report=classification_report(y_test, y_pred,output_dict=True)
  rfc_frame=pd.DataFrame.from_dict((report['macro avg']), orient='index').T
  rfc_frame['accuracy']=report['accuracy']
  rfc_frame['nombre']=name
  rfc_frame['True_neg']=confusion_matrix(y_test,y_pred)[0][0]
  rfc_frame['False_neg']=confusion_matrix(y_test,y_pred)[0][1]
  rfc_frame['False_posit']=confusion_matrix(y_test,y_pred)[1][0]
  rfc_frame['True_posit']=confusion_matrix(y_test,y_pred)[1][1]
  rfc_frame['Aveg_Precision-Recall']=average_precision_score(y_test, y_pred,average='macro')

  rfc_frame=rfc_frame.drop(columns=['support'])
  return rfc_frame

In [ ]:
evaluate_model('rf',y_out_sampe,y_pred, ypred_proba = None)

Accuracy of the model: 0.9999903842836206

Classification report: 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   5296756
           1       1.00      0.99      1.00      7061

    accuracy                           1.00   5303817
   macro avg       1.00      1.00      1.00   5303817
weighted avg       1.00      1.00      1.00   5303817


Confusion matrix: 
[[5296756       0]
 [     51    7010]]

Normalized confusion matrix: 
[[9.98668695e-01 0.00000000e+00]
 [9.61571638e-06 1.32168964e-03]]

Average Precision-Recall score:	 0.9927868427380477


,precision,recall,f1-score,accuracy,nombre,True_neg,False_neg,False_posit,True_posit,Aveg_Precision-Recall
0,0.999995,0.996389,0.998185,0.99999,rf,5296756,0,51,7010,0.992787


## Conclusión:

Al final mi conclusión es que de manera muy simplificada, esta práctica viene a reproducir lo que en realidad hacen las instituciones finacieras para minimizar sus perdidas por fraude en este tipo de transacciones. 

Por cierto, de absoluta actualidad en este último año a traves de SMS, llamadas falsas para hacarse con las contraseñas de banca movil de los clientes de los bancos, que todos hemos recibido.

Los departamentos de seguridad, tienen que actuar de manera muy rápida, cuando se produce una tipologia de fraude, ejemplo, compra con tarjeta en un página web de criptomonedas, bloquean esa operatoria. Puede que exista alguna transacción legitima (falso positivo) que quedaría sin realizarse, pero se evitan muchos falsos negativos, que son los que suponen un alto coste para la entidad.

Los estafadores, dejarán de utilizar esta operatoria, y comenzarán con otra que no se haya detectado. En nuestro ejemplo, en vez de dejar la cuenta a 0, dejar un saldo bajo. Los responsables de seguridad, modificarían la regla, y así se seguiría produciendo este juego del gato y el ratón.

En el mundo real, el machine learning ayuda a encontrar nuevas reglas, patrones, operatorias que a simple vista, no serían fáciles de detectar, y cada vez se van construyendo sistemas mas complejo de detección de fraude.
